In [1]:
import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.nn import Sequential,MaxPool2d,Linear,ReLU
from functions_pytorch import *
from pytorch_util import *
from pytorch_models import ConvBatchRelu
from albumentations_helper import create_transform
from albumentations import ShiftScaleRotate,Cutout,RandomContrast,RandomBrightness,Compose
import pickle
import numpy as np
import pretrainedmodels
from functools import partial

In [2]:
## configs ##
color = True
#shapes = (3,224,224)
HalfBatch = 16
outDim = 500
n_class = 5004
distanceFun = l2_distance
distanceFun_np = l2_distance_np
TTASize = 4
pct = 0.1

In [3]:
if color:
    with open('/home/will/Desktop/kaggle/Whale/train_df_color.pkl', 'rb') as f:
        Ids_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_train_color.pkl', 'rb') as f:
        newWhale_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/val_df_color.pkl', 'rb') as f:
        Ids_val = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_val_color.pkl', 'rb') as f:
        newWhale_val = pickle.load(f)
else:
    with open('/home/will/Desktop/kaggle/Whale/train_df.pkl', 'rb') as f:
        Ids_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_train.pkl', 'rb') as f:
        newWhale_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/val_df.pkl', 'rb') as f:
        Ids_val = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_val.pkl', 'rb') as f:
        newWhale_val = pickle.load(f)

In [4]:
# combine whales with one image with new whale
newWhale_train2 = newWhale_train + [i[0] for i in Ids_train.loc[Ids_train.length2 == 1].Imgs.tolist()]
Ids_train2 = Ids_train.loc[Ids_train.length2>1]

In [5]:
# parameters for adv training
k = 10
numBatch = 200
batchSize = 16
maxImg = Ids_train2.shape[0]-1

mapping_dict = dict(zip(Ids_train.Id.values,Ids_train.index.values))
labels = Ids_val.Id.map(mapping_dict)
aggFun = partial(np.quantile,q=pct,axis=(1,2))

Set up dataloader

In [6]:
aug = Compose([RandomContrast(p=0.5),RandomBrightness(p=0.5),
                ShiftScaleRotate(shift_limit=0.03,rotate_limit=25,scale_limit=0.05,p=1),Cutout(p=0.5)])
transform = create_transform(aug)  

aug_test = Compose([RandomContrast(p=0.2),RandomBrightness(p=0.2),
                ShiftScaleRotate(shift_limit=0.03,rotate_limit=15,scale_limit=0.02,p=1)])
transform_test = create_transform(aug_test)    

In [7]:
# gen_train = TripletGenerator(Ids_train,newWhale_train,transform,p=0.08)
# triplet_dl= DataLoader(gen_train,HalfBatch,True,num_workers=3,drop_last=True,worker_init_fn=worker_init_fn)
gen_train2 = softmaxGenerator(Ids_train,transform)
softmax_dl= DataLoader(gen_train2,HalfBatch,True,num_workers=3,drop_last=True,worker_init_fn=worker_init_fn)

Set up training

In [8]:
model = pretrainedmodels.densenet121()
model = fine_tune_pretrainedmodels(model, outDim).to('cuda:0')

In [10]:
# load model and opt
checkpoint = torch.load('Models/chk14_softmax')
model.load_state_dict(checkpoint['model_state_dict'])
#opt.load_state_dict(checkpoint['optimizer_state_dict'])

In [11]:
# validation MAP
pred_train_gen = PredictGenerator2(Ids_train.Imgs.tolist(),transform_test,TTASize, 4,True)
pred_val_gen = PredictGenerator2(Ids_val.Imgs.tolist(),transform_test,TTASize, 2,True)

feature_train = predict(model,pred_train_gen)
feature_val = predict(model,pred_val_gen)

feature_train = feature_train.reshape(Ids_train.shape[0],TTASize,outDim)
feature_val = feature_val.reshape(Ids_val.shape[0],TTASize,outDim)

predicts = loop_distance(feature_train,feature_val,distanceFun_np,aggFun,returnValue=False,k=5)

MAP(labels,predicts)

0.7392702535559679

In [13]:
# adv training
feature_train2 = feature_train[Ids_train.length2>1]
predicts2 = loop_distance(feature_train2,feature_train2,distanceFun_np,aggFun,returnValue=False,k=k)
distDict = {i:[j for j in item.tolist() if i!=j] for i,item in enumerate(predicts2)}
batch_sampler = AdvSample(distDict,numBatch,batchSize,maxImg)

gen_train = TripletGenerator(Ids_train2,newWhale_train2,transform)
triplet_dl = DataLoader(gen_train,num_workers=3,batch_sampler=batch_sampler)

In [14]:
basemodel = model.features

In [15]:
triplet_model = model

In [16]:
softmax_head = nn.Linear(model.last_linear.in_features,n_class,bias=False).to('cuda:0')

In [17]:
softmax_model = combineModel(basemodel,softmax_head).to('cuda:0')

#### first fix the base and fine-tune the softmax layer

In [18]:
set_requires_grad(softmax_model, False)
set_requires_grad(softmax_head, True)

In [19]:
opt = Adam(trainable_parameter(softmax_head),lr=1e-3)

In [20]:
softmax_ncls_loss = loss_func_generator_softmax_ncls()

In [21]:
np.log(1/5004)

-8.517992871586802

In [22]:
softmax_model = fit(5, softmax_model, softmax_ncls_loss, opt, softmax_dl,lossBest=None)

epoch:0, train_loss:8.552428557322575
epoch:1, train_loss:7.506294331489465
epoch:2, train_loss:6.27407548060784
epoch:3, train_loss:5.173435159218617
epoch:4, train_loss:4.180193519745117
Training completed in 55.15099024772644s


In [23]:
softmax_model = fit(3, softmax_model, softmax_ncls_loss, opt, softmax_dl,lossBest=None)

epoch:0, train_loss:3.3512251468805165
epoch:1, train_loss:2.713530243207247
epoch:2, train_loss:2.2012481479308543
Training completed in 32.968348264694214s


In [24]:
softmax_model = fit(3, softmax_model, softmax_ncls_loss, opt, softmax_dl,lossBest=None)

epoch:0, train_loss:1.799471650750209
epoch:1, train_loss:1.4987670462100933
epoch:2, train_loss:1.2952542012700667
Training completed in 33.062697887420654s


In [25]:
softmax_model = fit(1, softmax_model, softmax_ncls_loss, opt, softmax_dl,lossBest=None)

epoch:0, train_loss:1.1025530717884884
Training completed in 11.096718788146973s


#### iterate over softmax and triplet loss

In [26]:
set_requires_grad(softmax_model, True)
set_requires_grad(triplet_model, True)

In [32]:
temp = torch.ones((batchSize,batchSize),device='cuda:0') * 6
temp[torch.arange(batchSize),torch.arange(batchSize)] = 8.5
loss_func = loss_func_generator_softmax_Temp(batchSize,distanceFun,temp)

In [28]:
opt_triplet = Adam([
            {"params": gather_parameter_nameList(['denseblock4','norm5'],triplet_model), "lr": 1e-4},
            {"params": gather_parameter_nameList(['denseblock3','transition3'],triplet_model), "lr": 1e-5},
            {"params": gather_parameter_nameList(['denseblock2','transition2'],triplet_model), "lr": 1e-6},
            {"params": triplet_model.last_linear.parameters(), "lr": 2e-4}
            ])

In [29]:
opt_softmax = Adam([
            {"params": gather_parameter_nameList(['denseblock4','norm5'],softmax_model), "lr": 1e-4},
            {"params": gather_parameter_nameList(['denseblock3','transition3'],softmax_model), "lr": 1e-5},
            {"params": gather_parameter_nameList(['denseblock2','transition2'],softmax_model), "lr": 1e-6},
            {"params": softmax_model.linear.parameters(), "lr": 2e-4}
            ])

In [30]:
softmax_model = fit(1, softmax_model, softmax_ncls_loss, opt_softmax, softmax_dl,lossBest=None)

epoch:0, train_loss:0.9173421550255555
Training completed in 33.345388412475586s


In [33]:
triplet_model = fit(1, triplet_model, loss_func, opt_triplet, triplet_dl, lossBest=None)

epoch:0, train_loss:1.1794272028520458
Training completed in 39.54643416404724s


In [34]:
# validation MAP for softmax_model
pred_val_gen = PredictGenerator2(Ids_val.Imgs.tolist(),transform_test,TTASize, 2,True)
feature_val = predict(softmax_model,pred_val_gen)
feature_val = feature_val.reshape(Ids_val.shape[0],TTASize,n_class)
feature_val_mean = -feature_val.mean(1)
predict_softmax = np.array([top_k(x) for x in feature_val_mean])
MAP(labels,predicts)

0.7392702535559679

In [35]:
# validation MAP for triplet_model
pred_train_gen = PredictGenerator2(Ids_train.Imgs.tolist(),transform_test,TTASize, 4,True)
pred_val_gen = PredictGenerator2(Ids_val.Imgs.tolist(),transform_test,TTASize, 2,True)

feature_train = predict(triplet_model,pred_train_gen)
feature_val = predict(triplet_model,pred_val_gen)

feature_train = feature_train.reshape(Ids_train.shape[0],TTASize,outDim)
feature_val = feature_val.reshape(Ids_val.shape[0],TTASize,outDim)

predicts = loop_distance(feature_train,feature_val,distanceFun_np,aggFun,returnValue=False,k=5)

MAP(labels,predicts)

0.7620284477427335

In [36]:
torch.save({
            'model_state_dict': triplet_model.state_dict(),
            'optimizer_state_dict': opt_triplet.state_dict(),
            }, 'Models/chk1_triplet_model')
torch.save({
            'model_state_dict': softmax_model.state_dict(),
            'optimizer_state_dict': opt_softmax.state_dict(),
            }, 'Models/chk1_softmax_model')

In [37]:
# adv training
feature_train2 = feature_train[Ids_train.length2>1]
predicts2 = loop_distance(feature_train2,feature_train2,distanceFun_np,aggFun,returnValue=False,k=k)
distDict = {i:[j for j in item.tolist() if i!=j] for i,item in enumerate(predicts2)}
batch_sampler = AdvSample(distDict,numBatch,batchSize,maxImg)

gen_train = TripletGenerator(Ids_train2,newWhale_train2,transform)
triplet_dl = DataLoader(gen_train,num_workers=3,batch_sampler=batch_sampler)

In [217]:
# checkpoint = torch.load('Models/chk1_triplet_model')
# triplet_model.load_state_dict(checkpoint['model_state_dict'])
# opt_triplet.load_state_dict(checkpoint['optimizer_state_dict'])

# checkpoint = torch.load('Models/chk1_softmax_model')
# softmax_model.load_state_dict(checkpoint['model_state_dict'])
# opt_softmax.load_state_dict(checkpoint['optimizer_state_dict'])

In [38]:
triplet_model = fit(1, triplet_model, loss_func, opt_triplet, triplet_dl, lossBest=None)

epoch:0, train_loss:1.0152618999038239
Training completed in 39.425015687942505s


In [39]:
softmax_model = fit(1, softmax_model, softmax_ncls_loss, opt_softmax, softmax_dl,lossBest=None)

epoch:0, train_loss:0.7634027809477769
Training completed in 33.31382513046265s


In [40]:
triplet_model = fit(1, triplet_model, loss_func, opt_triplet, triplet_dl, lossBest=None)

epoch:0, train_loss:1.0172637203514903
Training completed in 39.62376570701599s


In [41]:
softmax_model = fit(1, softmax_model, softmax_ncls_loss, opt_softmax, softmax_dl,lossBest=None)

epoch:0, train_loss:0.6760807453821867
Training completed in 33.42255353927612s


In [42]:
# validation MAP for softmax_model
pred_val_gen = PredictGenerator2(Ids_val.Imgs.tolist(),transform_test,TTASize, 2,True)
feature_val = predict(softmax_model,pred_val_gen)
feature_val = feature_val.reshape(Ids_val.shape[0],TTASize,n_class)
feature_val_mean = -feature_val.mean(1)
predict_softmax = np.array([top_k(x) for x in feature_val_mean])
MAP(labels,predicts)

0.7620284477427335

In [43]:
# validation MAP for triplet_model
pred_train_gen = PredictGenerator2(Ids_train.Imgs.tolist(),transform_test,TTASize, 4,True)
pred_val_gen = PredictGenerator2(Ids_val.Imgs.tolist(),transform_test,TTASize, 2,True)

feature_train = predict(triplet_model,pred_train_gen)
feature_val = predict(triplet_model,pred_val_gen)

feature_train = feature_train.reshape(Ids_train.shape[0],TTASize,outDim)
feature_val = feature_val.reshape(Ids_val.shape[0],TTASize,outDim)

predicts = loop_distance(feature_train,feature_val,distanceFun_np,aggFun,returnValue=False,k=5)

MAP(labels,predicts)

0.7730055658627086

In [44]:
torch.save({
            'model_state_dict': triplet_model.state_dict(),
            'optimizer_state_dict': opt_triplet.state_dict(),
            }, 'Models/chk2_triplet_model')
torch.save({
            'model_state_dict': softmax_model.state_dict(),
            'optimizer_state_dict': opt_softmax.state_dict(),
            }, 'Models/chk2_softmax_model')

In [45]:
# adv training
feature_train2 = feature_train[Ids_train.length2>1]
predicts2 = loop_distance(feature_train2,feature_train2,distanceFun_np,aggFun,returnValue=False,k=k)
distDict = {i:[j for j in item.tolist() if i!=j] for i,item in enumerate(predicts2)}
batch_sampler = AdvSample(distDict,numBatch,batchSize,maxImg)

gen_train = TripletGenerator(Ids_train2,newWhale_train2,transform)
triplet_dl = DataLoader(gen_train,num_workers=3,batch_sampler=batch_sampler)

In [46]:
triplet_model = fit(1, triplet_model, loss_func, opt_triplet, triplet_dl, lossBest=None)

epoch:0, train_loss:1.0130826754729605
Training completed in 39.5515878200531s


In [47]:
softmax_model = fit(1, softmax_model, softmax_ncls_loss, opt_softmax, softmax_dl,lossBest=None)

epoch:0, train_loss:0.60712833253619
Training completed in 33.97565770149231s


In [48]:
triplet_model = fit(1, triplet_model, loss_func, opt_triplet, triplet_dl, lossBest=None)

epoch:0, train_loss:0.8980482210878467
Training completed in 39.56730794906616s


In [49]:
softmax_model = fit(1, softmax_model, softmax_ncls_loss, opt_softmax, softmax_dl,lossBest=None)

epoch:0, train_loss:0.5547530082746958
Training completed in 33.485716104507446s


In [50]:
triplet_model = fit(1, triplet_model, loss_func, opt_triplet, triplet_dl, lossBest=None)

epoch:0, train_loss:0.9432387741211333
Training completed in 39.566426038742065s


In [51]:
softmax_model = fit(1, softmax_model, softmax_ncls_loss, opt_softmax, softmax_dl,lossBest=None)

epoch:0, train_loss:0.49342067042986554
Training completed in 33.392844676971436s


In [52]:
triplet_model = fit(1, triplet_model, loss_func, opt_triplet, triplet_dl, lossBest=None,clip=4)

epoch:0, train_loss:0.9402216973363376
Training completed in 40.291356325149536s


In [53]:
softmax_model = fit(1, softmax_model, softmax_ncls_loss, opt_softmax, softmax_dl,lossBest=None,clip=4)

epoch:0, train_loss:0.44436025170561594
Training completed in 34.421178102493286s


In [54]:
# validation MAP for softmax_model
pred_val_gen = PredictGenerator2(Ids_val.Imgs.tolist(),transform_test,TTASize, 2,True)
feature_val = predict(softmax_model,pred_val_gen)
feature_val = feature_val.reshape(Ids_val.shape[0],TTASize,n_class)
feature_val_mean = -feature_val.mean(1)
predict_softmax = np.array([top_k(x) for x in feature_val_mean])
MAP(labels,predicts)

0.7730055658627086

In [55]:
# validation MAP for triplet_model
pred_train_gen = PredictGenerator2(Ids_train.Imgs.tolist(),transform_test,TTASize, 4,True)
pred_val_gen = PredictGenerator2(Ids_val.Imgs.tolist(),transform_test,TTASize, 2,True)

feature_train = predict(triplet_model,pred_train_gen)
feature_val = predict(triplet_model,pred_val_gen)

feature_train = feature_train.reshape(Ids_train.shape[0],TTASize,outDim)
feature_val = feature_val.reshape(Ids_val.shape[0],TTASize,outDim)

predicts = loop_distance(feature_train,feature_val,distanceFun_np,aggFun,returnValue=False,k=5)

MAP(labels,predicts)

0.7804421768707482

In [56]:
torch.save({
            'model_state_dict': triplet_model.state_dict(),
            'optimizer_state_dict': opt_triplet.state_dict(),
            }, 'Models/chk3_triplet_model')
torch.save({
            'model_state_dict': softmax_model.state_dict(),
            'optimizer_state_dict': opt_softmax.state_dict(),
            }, 'Models/chk3_softmax_model')

In [57]:
# adv training
feature_train2 = feature_train[Ids_train.length2>1]
predicts2 = loop_distance(feature_train2,feature_train2,distanceFun_np,aggFun,returnValue=False,k=k)
distDict = {i:[j for j in item.tolist() if i!=j] for i,item in enumerate(predicts2)}
batch_sampler = AdvSample(distDict,numBatch,batchSize,maxImg)

gen_train = TripletGenerator(Ids_train2,newWhale_train2,transform)
triplet_dl = DataLoader(gen_train,num_workers=3,batch_sampler=batch_sampler)

In [58]:
triplet_model = fit(1, triplet_model, loss_func, opt_triplet, triplet_dl, lossBest=None)

epoch:0, train_loss:0.9602008053188116
Training completed in 39.59159755706787s


In [59]:
softmax_model = fit(1, softmax_model, softmax_ncls_loss, opt_softmax, softmax_dl,lossBest=None)

epoch:0, train_loss:0.4137777493168146
Training completed in 33.49457621574402s


In [60]:
triplet_model = fit(1, triplet_model, loss_func, opt_triplet, triplet_dl, lossBest=None)

epoch:0, train_loss:0.931597711732153
Training completed in 39.61746287345886s


In [61]:
softmax_model = fit(1, softmax_model, softmax_ncls_loss, opt_softmax, softmax_dl,lossBest=None)

epoch:0, train_loss:0.3701094229442951
Training completed in 33.44179344177246s


In [62]:
triplet_model = fit(1, triplet_model, loss_func, opt_triplet, triplet_dl, lossBest=None)

epoch:0, train_loss:0.8817506249729401
Training completed in 39.64719605445862s


In [63]:
softmax_model = fit(1, softmax_model, softmax_ncls_loss, opt_softmax, softmax_dl,lossBest=None)

epoch:0, train_loss:0.32968752401379436
Training completed in 33.62010383605957s


In [64]:
triplet_model = fit(1, triplet_model, loss_func, opt_triplet, triplet_dl, lossBest=None,clip=4)

epoch:0, train_loss:0.8879198306570939
Training completed in 40.27020764350891s


In [65]:
softmax_model = fit(1, softmax_model, softmax_ncls_loss, opt_softmax, softmax_dl,lossBest=None,clip=4)

epoch:0, train_loss:0.32642006205442625
Training completed in 34.49658727645874s


In [66]:
# validation MAP for softmax_model
pred_val_gen = PredictGenerator2(Ids_val.Imgs.tolist(),transform_test,TTASize, 2,True)
feature_val = predict(softmax_model,pred_val_gen)
feature_val = feature_val.reshape(Ids_val.shape[0],TTASize,n_class)
feature_val_mean = -feature_val.mean(1)
predict_softmax = np.array([top_k(x) for x in feature_val_mean])
MAP(labels,predicts)

0.7804421768707482

In [67]:
# validation MAP for triplet_model
pred_train_gen = PredictGenerator2(Ids_train.Imgs.tolist(),transform_test,TTASize, 4,True)
pred_val_gen = PredictGenerator2(Ids_val.Imgs.tolist(),transform_test,TTASize, 2,True)

feature_train = predict(triplet_model,pred_train_gen)
feature_val = predict(triplet_model,pred_val_gen)

feature_train = feature_train.reshape(Ids_train.shape[0],TTASize,outDim)
feature_val = feature_val.reshape(Ids_val.shape[0],TTASize,outDim)

predicts = loop_distance(feature_train,feature_val,distanceFun_np,aggFun,returnValue=False,k=5)

MAP(labels,predicts)

0.7854669140383427

In [68]:
torch.save({
            'model_state_dict': triplet_model.state_dict(),
            'optimizer_state_dict': opt_triplet.state_dict(),
            }, 'Models/chk4_triplet_model')
torch.save({
            'model_state_dict': softmax_model.state_dict(),
            'optimizer_state_dict': opt_softmax.state_dict(),
            }, 'Models/chk4_softmax_model')

# the rest is copied over from Training-softmaxLoss-pytorch-Adv-withTemp

Fine-tune previous layers

In [ ]:
# for ResNet50
# set_requires_grad([model[1].layer4,model[1].layer3,model[1].layer2],True)
# opt = Adam([
#             {"params": model[1].layer4.parameters(), "lr": 1e-5},
#             {"params": model[1].layer3.parameters(), "lr": 1e-6},
#             {"params": model[1].layer2.parameters(), "lr": 5e-7},
#             {"params": model[1].last_linear.parameters(), "lr": 2e-5},
#             {"params": model[0].parameters(), "lr": 2e-5}
#             ])

In [15]:
# for DenseNet121
name_list = ['denseblock3','transition3','denseblock4','norm5']
set_requires_grad_paraList(gather_parameter_nameList(name_list,model),True)

In [16]:
opt = Adam([
            {"params": gather_parameter_nameList(['denseblock4','norm5'],model), "lr": 1e-4},
            {"params": gather_parameter_nameList(['denseblock3','transition3'],model), "lr": 1e-5},
            {"params": gather_parameter_nameList(['denseblock2','transition2'],model), "lr": 1e-6},
            {"params": model.last_linear.parameters(), "lr": 2e-4}
            ])

In [13]:
model = fit(5, model, loss_func, opt, train_dl, valid_dl,lossBest=None)

epoch:0, train_loss:0.7810285010787307, val_loss:0.3778223693370819
epoch:1, train_loss:0.5369224858780702, val_loss:0.29206743836402893
epoch:2, train_loss:0.4689952476062084, val_loss:0.31415751576423645
epoch:3, train_loss:0.3900194559348086, val_loss:0.2408030480146408
epoch:4, train_loss:0.37403056503889337, val_loss:0.23127534985542297
Training completed in 163.73842453956604s


In [14]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            }, 'Models/chk1_softmax')

In [19]:
# parameters
k = 50
numBatch = 200
batchSize = 16
maxImg = Ids_train2.shape[0]-1

mapping_dict = dict(zip(Ids_train.Id.values,Ids_train.index.values))
labels = Ids_val.Id.map(mapping_dict)
aggFun = partial(np.quantile,q=pct,axis=(1,2))

In [17]:
# validation MAP
pred_train_gen = PredictGenerator2(Ids_train.Imgs.tolist(),transform_test,TTASize, 4,True)
pred_val_gen = PredictGenerator2(Ids_val.Imgs.tolist(),transform_test,TTASize, 2,True)

feature_train = predict(model,pred_train_gen)
feature_val = predict(model,pred_val_gen)

feature_train = feature_train.reshape(Ids_train.shape[0],TTASize,outDim)
feature_val = feature_val.reshape(Ids_val.shape[0],TTASize,outDim)

predicts = loop_distance(feature_train,feature_val,distanceFun_np,aggFun,returnValue=False,k=5)

MAP(labels,predicts)

0.24036796536796537

In [18]:
# adv training
feature_train2 = feature_train[Ids_train.length2>1]
predicts2 = loop_distance(feature_train2,feature_train2,distanceFun_np,aggFun,returnValue=False,k=k)
distDict = {i:[j for j in item.tolist() if i!=j] for i,item in enumerate(predicts2)}
batch_sampler = AdvSample(distDict,numBatch,batchSize,maxImg)

gen_train = TripletGenerator(Ids_train2,newWhale_train2,transform)
train_dl= DataLoader(gen_train,num_workers=3,batch_sampler=batch_sampler)

In [19]:
loss_func = loss_func_generator_softmax_Temp(batchSize,distanceFun,0.5)

In [20]:
model = fit(5, model, loss_func, opt, train_dl, lossBest=None)

epoch:0, train_loss:1.6357341796322598
epoch:1, train_loss:1.4789364595230812
epoch:2, train_loss:1.3506384879513516
epoch:3, train_loss:1.3398994163411562
epoch:4, train_loss:1.2064544385899612
Training completed in 110.71649432182312s


In [21]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            }, 'Models/chk2_softmax')

In [22]:
# validation MAP
pred_train_gen = PredictGenerator2(Ids_train.Imgs.tolist(),transform_test,TTASize, 4,True)
pred_val_gen = PredictGenerator2(Ids_val.Imgs.tolist(),transform_test,TTASize, 2,True)

feature_train = predict(model,pred_train_gen)
feature_val = predict(model,pred_val_gen)

feature_train = feature_train.reshape(Ids_train.shape[0],TTASize,outDim)
feature_val = feature_val.reshape(Ids_val.shape[0],TTASize,outDim)

predicts = loop_distance(feature_train,feature_val,distanceFun_np,aggFun,returnValue=False,k=5)

MAP(labels,predicts)

0.3641774891774892

In [23]:
# adv training
feature_train2 = feature_train[Ids_train.length2>1]
predicts2 = loop_distance(feature_train2,feature_train2,distanceFun_np,aggFun,returnValue=False,k=k)
distDict = {i:[j for j in item.tolist() if i!=j] for i,item in enumerate(predicts2)}
batch_sampler = AdvSample(distDict,numBatch,batchSize,maxImg)

gen_train = TripletGenerator(Ids_train2,newWhale_train2,transform)
train_dl= DataLoader(gen_train,num_workers=3,batch_sampler=batch_sampler)

In [24]:
loss_func = loss_func_generator_softmax_Temp(batchSize,distanceFun,1)

In [25]:
model = fit(5, model, loss_func, opt, train_dl, lossBest=None)

epoch:0, train_loss:1.2777612315582447
epoch:1, train_loss:1.042473152524135
epoch:2, train_loss:1.053432722560695
epoch:3, train_loss:0.9402261529463888
epoch:4, train_loss:0.9117436111380494
Training completed in 110.93116331100464s


In [26]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            }, 'Models/chk3_softmax')

In [27]:
# validation MAP
pred_train_gen = PredictGenerator2(Ids_train.Imgs.tolist(),transform_test,TTASize, 4,True)
pred_val_gen = PredictGenerator2(Ids_val.Imgs.tolist(),transform_test,TTASize, 2,True)

feature_train = predict(model,pred_train_gen)
feature_val = predict(model,pred_val_gen)

feature_train = feature_train.reshape(Ids_train.shape[0],TTASize,outDim)
feature_val = feature_val.reshape(Ids_val.shape[0],TTASize,outDim)

predicts = loop_distance(feature_train,feature_val,distanceFun_np,aggFun,returnValue=False,k=5)

MAP(labels,predicts)

0.4340136054421769

In [28]:
# adv training
feature_train2 = feature_train[Ids_train.length2>1]
predicts2 = loop_distance(feature_train2,feature_train2,distanceFun_np,aggFun,returnValue=False,k=k)
distDict = {i:[j for j in item.tolist() if i!=j] for i,item in enumerate(predicts2)}
batch_sampler = AdvSample(distDict,numBatch,batchSize,maxImg)

gen_train = TripletGenerator(Ids_train2,newWhale_train2,transform)
train_dl= DataLoader(gen_train,num_workers=3,batch_sampler=batch_sampler)

In [29]:
loss_func = loss_func_generator_softmax_Temp(batchSize,distanceFun,1)

In [30]:
model = fit(5, model, loss_func, opt, train_dl, lossBest=None)

epoch:0, train_loss:0.9008529884079115
epoch:1, train_loss:0.8060934820461795
epoch:2, train_loss:0.8383919769704667
epoch:3, train_loss:0.7787562650246699
epoch:4, train_loss:0.7128494815503965
Training completed in 110.94993877410889s


In [31]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            }, 'Models/chk4_softmax')

In [32]:
# validation MAP
pred_train_gen = PredictGenerator2(Ids_train.Imgs.tolist(),transform_test,TTASize, 4,True)
pred_val_gen = PredictGenerator2(Ids_val.Imgs.tolist(),transform_test,TTASize, 2,True)

feature_train = predict(model,pred_train_gen)
feature_val = predict(model,pred_val_gen)

feature_train = feature_train.reshape(Ids_train.shape[0],TTASize,outDim)
feature_val = feature_val.reshape(Ids_val.shape[0],TTASize,outDim)

predicts = loop_distance(feature_train,feature_val,distanceFun_np,aggFun,returnValue=False,k=5)

MAP(labels,predicts)

0.49706246134817567

In [33]:
# adv training
feature_train2 = feature_train[Ids_train.length2>1]
predicts2 = loop_distance(feature_train2,feature_train2,distanceFun_np,aggFun,returnValue=False,k=k)
distDict = {i:[j for j in item.tolist() if i!=j] for i,item in enumerate(predicts2)}
batch_sampler = AdvSample(distDict,numBatch,batchSize,maxImg)

gen_train = TripletGenerator(Ids_train2,newWhale_train2,transform)
train_dl= DataLoader(gen_train,num_workers=3,batch_sampler=batch_sampler)

In [34]:
loss_func = loss_func_generator_softmax_Temp(batchSize,distanceFun,1)

In [35]:
model = fit(5, model, loss_func, opt, train_dl, lossBest=None)

epoch:0, train_loss:0.7866094325328133
epoch:1, train_loss:0.7456951266755172
epoch:2, train_loss:0.7163024214528949
epoch:3, train_loss:0.680116604911825
epoch:4, train_loss:0.636626806537636
Training completed in 110.68660593032837s


In [36]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            }, 'Models/chk5_softmax')

In [37]:
k = 30

In [38]:
# validation MAP
pred_train_gen = PredictGenerator2(Ids_train.Imgs.tolist(),transform_test,TTASize, 4,True)
pred_val_gen = PredictGenerator2(Ids_val.Imgs.tolist(),transform_test,TTASize, 2,True)

feature_train = predict(model,pred_train_gen)
feature_val = predict(model,pred_val_gen)

feature_train = feature_train.reshape(Ids_train.shape[0],TTASize,outDim)
feature_val = feature_val.reshape(Ids_val.shape[0],TTASize,outDim)

predicts = loop_distance(feature_train,feature_val,distanceFun_np,aggFun,returnValue=False,k=5)

MAP(labels,predicts)

0.5329313543599258

In [39]:
# adv training
feature_train2 = feature_train[Ids_train.length2>1]
predicts2 = loop_distance(feature_train2,feature_train2,distanceFun_np,aggFun,returnValue=False,k=k)
distDict = {i:[j for j in item.tolist() if i!=j] for i,item in enumerate(predicts2)}
batch_sampler = AdvSample(distDict,numBatch,batchSize,maxImg)

gen_train = TripletGenerator(Ids_train2,newWhale_train2,transform)
train_dl= DataLoader(gen_train,num_workers=3,batch_sampler=batch_sampler)

In [40]:
loss_func = loss_func_generator_softmax_Temp(batchSize,distanceFun,2)

In [41]:
model = fit(5, model, loss_func, opt, train_dl, lossBest=None)

epoch:0, train_loss:0.8808728531817269
epoch:1, train_loss:0.8264410771388825
epoch:2, train_loss:0.7651666348260608
epoch:3, train_loss:0.642489023628782
epoch:4, train_loss:0.6516656506338406
Training completed in 113.60318064689636s


In [42]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            }, 'Models/chk5_softmax')

In [43]:
# validation MAP
pred_train_gen = PredictGenerator2(Ids_train.Imgs.tolist(),transform_test,TTASize, 4,True)
pred_val_gen = PredictGenerator2(Ids_val.Imgs.tolist(),transform_test,TTASize, 2,True)

feature_train = predict(model,pred_train_gen)
feature_val = predict(model,pred_val_gen)

feature_train = feature_train.reshape(Ids_train.shape[0],TTASize,outDim)
feature_val = feature_val.reshape(Ids_val.shape[0],TTASize,outDim)

predicts = loop_distance(feature_train,feature_val,distanceFun_np,aggFun,returnValue=False,k=5)

MAP(labels,predicts)

0.5610235003092146

In [44]:
# adv training
feature_train2 = feature_train[Ids_train.length2>1]
predicts2 = loop_distance(feature_train2,feature_train2,distanceFun_np,aggFun,returnValue=False,k=k)
distDict = {i:[j for j in item.tolist() if i!=j] for i,item in enumerate(predicts2)}
batch_sampler = AdvSample(distDict,numBatch,batchSize,maxImg)

gen_train = TripletGenerator(Ids_train2,newWhale_train2,transform)
train_dl= DataLoader(gen_train,num_workers=3,batch_sampler=batch_sampler)

In [45]:
loss_func = loss_func_generator_softmax_Temp(batchSize,distanceFun,2)

In [46]:
model = fit(5, model, loss_func, opt, train_dl, lossBest=None)

epoch:0, train_loss:0.6887034554787672
epoch:1, train_loss:0.6549804205777215
epoch:2, train_loss:0.6540592023746563
epoch:3, train_loss:0.6092244425757987
epoch:4, train_loss:0.6051490222577189
Training completed in 111.02671241760254s


In [47]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            }, 'Models/chk6_softmax')

In [48]:
k=20

In [49]:
# validation MAP
pred_train_gen = PredictGenerator2(Ids_train.Imgs.tolist(),transform_test,TTASize, 4,True)
pred_val_gen = PredictGenerator2(Ids_val.Imgs.tolist(),transform_test,TTASize, 2,True)

feature_train = predict(model,pred_train_gen)
feature_val = predict(model,pred_val_gen)

feature_train = feature_train.reshape(Ids_train.shape[0],TTASize,outDim)
feature_val = feature_val.reshape(Ids_val.shape[0],TTASize,outDim)

predicts = loop_distance(feature_train,feature_val,distanceFun_np,aggFun,returnValue=False,k=5)

MAP(labels,predicts)

0.5879870129870131

In [50]:
# adv training
feature_train2 = feature_train[Ids_train.length2>1]
predicts2 = loop_distance(feature_train2,feature_train2,distanceFun_np,aggFun,returnValue=False,k=k)
distDict = {i:[j for j in item.tolist() if i!=j] for i,item in enumerate(predicts2)}
batch_sampler = AdvSample(distDict,numBatch,batchSize,maxImg)

gen_train = TripletGenerator(Ids_train2,newWhale_train2,transform)
train_dl= DataLoader(gen_train,num_workers=3,batch_sampler=batch_sampler)

In [51]:
loss_func = loss_func_generator_softmax_Temp(batchSize,distanceFun,4)

In [52]:
model = fit(5, model, loss_func, opt, train_dl, lossBest=None)

epoch:0, train_loss:0.9244699263588978
epoch:1, train_loss:0.7520761778527271
epoch:2, train_loss:0.6808652697039432
epoch:3, train_loss:0.6345080832233194
epoch:4, train_loss:0.6399718110190064
Training completed in 110.97688150405884s


In [53]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            }, 'Models/chk7_softmax')

In [79]:
k=20

In [80]:
# validation MAP
pred_train_gen = PredictGenerator2(Ids_train.Imgs.tolist(),transform_test,TTASize, 4,True)
pred_val_gen = PredictGenerator2(Ids_val.Imgs.tolist(),transform_test,TTASize, 2,True)

feature_train = predict(model,pred_train_gen)
feature_val = predict(model,pred_val_gen)

feature_train = feature_train.reshape(Ids_train.shape[0],TTASize,outDim)
feature_val = feature_val.reshape(Ids_val.shape[0],TTASize,outDim)

predicts = loop_distance(feature_train,feature_val,distanceFun_np,aggFun,returnValue=False,k=5)

MAP(labels,predicts)

0.6138991960420531

In [81]:
# adv training
feature_train2 = feature_train[Ids_train.length2>1]
predicts2 = loop_distance(feature_train2,feature_train2,distanceFun_np,aggFun,returnValue=False,k=k)
distDict = {i:[j for j in item.tolist() if i!=j] for i,item in enumerate(predicts2)}
batch_sampler = AdvSample(distDict,numBatch,batchSize,maxImg)

gen_train = TripletGenerator(Ids_train2,newWhale_train2,transform)
train_dl= DataLoader(gen_train,num_workers=3,batch_sampler=batch_sampler)

In [24]:
temp = torch.ones((batchSize,batchSize),device='cuda:0') * 4
temp[torch.arange(batchSize),torch.arange(batchSize)] = 5

In [83]:
loss_func = loss_func_generator_softmax_Temp(batchSize,distanceFun,temp)

In [84]:
model = fit(5, model, loss_func, opt, train_dl, lossBest=None)

epoch:0, train_loss:1.14703390246532
epoch:1, train_loss:1.0281447813810547
epoch:2, train_loss:1.0145563427867785
epoch:3, train_loss:0.9966852046101471
epoch:4, train_loss:0.9258869370964707
Training completed in 111.22468185424805s


In [85]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            }, 'Models/chk8_softmax')

In [23]:
k=10

In [22]:
# validation MAP
pred_train_gen = PredictGenerator2(Ids_train.Imgs.tolist(),transform_test,TTASize, 4,True)
pred_val_gen = PredictGenerator2(Ids_val.Imgs.tolist(),transform_test,TTASize, 2,True)

feature_train = predict(model,pred_train_gen)
feature_val = predict(model,pred_val_gen)

feature_train = feature_train.reshape(Ids_train.shape[0],TTASize,outDim)
feature_val = feature_val.reshape(Ids_val.shape[0],TTASize,outDim)

predicts = loop_distance(feature_train,feature_val,distanceFun_np,aggFun,returnValue=False,k=5)

MAP(labels,predicts)

0.6908008658008659

In [105]:
# adv training
feature_train2 = feature_train[Ids_train.length2>1]
predicts2 = loop_distance(feature_train2,feature_train2,distanceFun_np,aggFun,returnValue=False,k=k)
distDict = {i:[j for j in item.tolist() if i!=j] for i,item in enumerate(predicts2)}
batch_sampler = AdvSample(distDict,numBatch,batchSize,maxImg)

gen_train = TripletGenerator(Ids_train2,newWhale_train2,transform)
train_dl= DataLoader(gen_train,num_workers=3,batch_sampler=batch_sampler)

In [106]:
loss_func = loss_func_generator_softmax_Temp(batchSize,distanceFun,temp)

In [107]:
model = fit(5, model, loss_func, opt, train_dl, lossBest=None)

epoch:0, train_loss:1.1533960927216733
epoch:1, train_loss:1.0611220960571466
epoch:2, train_loss:1.0241561054857702
epoch:3, train_loss:1.0516141748330632
epoch:4, train_loss:1.01720767399001
Training completed in 110.9772777557373s


In [108]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            }, 'Models/chk9_softmax')

In [109]:
k=10

In [110]:
# validation MAP
pred_train_gen = PredictGenerator2(Ids_train.Imgs.tolist(),transform_test,TTASize, 4,True)
pred_val_gen = PredictGenerator2(Ids_val.Imgs.tolist(),transform_test,TTASize, 2,True)

feature_train = predict(model,pred_train_gen)
feature_val = predict(model,pred_val_gen)

feature_train = feature_train.reshape(Ids_train.shape[0],TTASize,outDim)
feature_val = feature_val.reshape(Ids_val.shape[0],TTASize,outDim)

predicts = loop_distance(feature_train,feature_val,distanceFun_np,aggFun,returnValue=False,k=5)

MAP(labels,predicts)

0.6676716141001855

In [25]:
# adv training
feature_train2 = feature_train[Ids_train.length2>1]
predicts2 = loop_distance(feature_train2,feature_train2,distanceFun_np,aggFun,returnValue=False,k=k)
distDict = {i:[j for j in item.tolist() if i!=j] for i,item in enumerate(predicts2)}
batch_sampler = AdvSample(distDict,numBatch,batchSize,maxImg)

gen_train = TripletGenerator(Ids_train2,newWhale_train2,transform)
train_dl= DataLoader(gen_train,num_workers=3,batch_sampler=batch_sampler)

In [26]:
temp = torch.ones((batchSize,batchSize),device='cuda:0') * 6
temp[torch.arange(batchSize),torch.arange(batchSize)] = 8

In [27]:
loss_func = loss_func_generator_softmax_Temp(batchSize,distanceFun,temp)

In [28]:
model = fit(5, model, loss_func, opt, train_dl, lossBest=None)

epoch:0, train_loss:1.1138147467472514
epoch:1, train_loss:1.0732262225750366
epoch:2, train_loss:1.0917892705221646
epoch:3, train_loss:1.0086230331105612
epoch:4, train_loss:0.9855393247851909
Training completed in 111.1889820098877s


In [30]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            }, 'Models/chk10_softmax')

In [116]:
k=10

In [29]:
# validation MAP
pred_train_gen = PredictGenerator2(Ids_train.Imgs.tolist(),transform_test,TTASize, 4,True)
pred_val_gen = PredictGenerator2(Ids_val.Imgs.tolist(),transform_test,TTASize, 2,True)

feature_train = predict(model,pred_train_gen)
feature_val = predict(model,pred_val_gen)

feature_train = feature_train.reshape(Ids_train.shape[0],TTASize,outDim)
feature_val = feature_val.reshape(Ids_val.shape[0],TTASize,outDim)

predicts = loop_distance(feature_train,feature_val,distanceFun_np,aggFun,returnValue=False,k=5)

MAP(labels,predicts)

0.700278293135436

In [31]:
# adv training
feature_train2 = feature_train[Ids_train.length2>1]
predicts2 = loop_distance(feature_train2,feature_train2,distanceFun_np,aggFun,returnValue=False,k=k)
distDict = {i:[j for j in item.tolist() if i!=j] for i,item in enumerate(predicts2)}
batch_sampler = AdvSample(distDict,numBatch,batchSize,maxImg)

gen_train = TripletGenerator(Ids_train2,newWhale_train2,transform)
train_dl= DataLoader(gen_train,num_workers=3,batch_sampler=batch_sampler)

In [32]:
loss_func = loss_func_generator_softmax_Temp(batchSize,distanceFun,temp)

In [33]:
model = fit(5, model, loss_func, opt, train_dl, lossBest=None)

epoch:0, train_loss:1.0715792899737593
epoch:1, train_loss:0.9973182217345211
epoch:2, train_loss:1.0252600699826016
epoch:3, train_loss:0.9855394455008819
epoch:4, train_loss:0.9961773318345429
Training completed in 111.08159947395325s


In [38]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            }, 'Models/chk11_softmax')

In [116]:
k=10

In [34]:
# validation MAP
pred_train_gen = PredictGenerator2(Ids_train.Imgs.tolist(),transform_test,TTASize, 4,True)
pred_val_gen = PredictGenerator2(Ids_val.Imgs.tolist(),transform_test,TTASize, 2,True)

feature_train = predict(model,pred_train_gen)
feature_val = predict(model,pred_val_gen)

feature_train = feature_train.reshape(Ids_train.shape[0],TTASize,outDim)
feature_val = feature_val.reshape(Ids_val.shape[0],TTASize,outDim)

predicts = loop_distance(feature_train,feature_val,distanceFun_np,aggFun,returnValue=False,k=5)

MAP(labels,predicts)

0.7195887445887447

In [35]:
# adv training
feature_train2 = feature_train[Ids_train.length2>1]
predicts2 = loop_distance(feature_train2,feature_train2,distanceFun_np,aggFun,returnValue=False,k=k)
distDict = {i:[j for j in item.tolist() if i!=j] for i,item in enumerate(predicts2)}
batch_sampler = AdvSample(distDict,numBatch,batchSize,maxImg)

gen_train = TripletGenerator(Ids_train2,newWhale_train2,transform)
train_dl= DataLoader(gen_train,num_workers=3,batch_sampler=batch_sampler)

In [36]:
loss_func = loss_func_generator_softmax_Temp(batchSize,distanceFun,temp)

In [37]:
model = fit(5, model, loss_func, opt, train_dl, lossBest=None)

epoch:0, train_loss:1.0243798535866815
epoch:1, train_loss:0.974644305927506
epoch:2, train_loss:0.9019275261241881
epoch:3, train_loss:0.9820473317891523
epoch:4, train_loss:0.9574115609780687
Training completed in 111.39126515388489s


In [40]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            }, 'Models/chk12_softmax')

In [116]:
k=10

In [50]:
# validation MAP
pred_train_gen = PredictGenerator2(Ids_train.Imgs.tolist(),transform_test,TTASize, 4,True)
pred_val_gen = PredictGenerator2(Ids_val.Imgs.tolist(),transform_test,TTASize, 2,True)

feature_train = predict(model,pred_train_gen)
feature_val = predict(model,pred_val_gen)

feature_train = feature_train.reshape(Ids_train.shape[0],TTASize,outDim)
feature_val = feature_val.reshape(Ids_val.shape[0],TTASize,outDim)

predicts = loop_distance(feature_train,feature_val,distanceFun_np,aggFun,returnValue=False,k=5)

MAP(labels,predicts)

0.7326530612244897

In [51]:
temp = torch.ones((batchSize,batchSize),device='cuda:0') * 6
temp[torch.arange(batchSize),torch.arange(batchSize)] = 8.5

In [52]:
# adv training
feature_train2 = feature_train[Ids_train.length2>1]
predicts2 = loop_distance(feature_train2,feature_train2,distanceFun_np,aggFun,returnValue=False,k=k)
distDict = {i:[j for j in item.tolist() if i!=j] for i,item in enumerate(predicts2)}
batch_sampler = AdvSample(distDict,numBatch,batchSize,maxImg)

gen_train = TripletGenerator(Ids_train2,newWhale_train2,transform)
train_dl= DataLoader(gen_train,num_workers=3,batch_sampler=batch_sampler)

In [53]:
loss_func = loss_func_generator_softmax_Temp(batchSize,distanceFun,temp)

In [54]:
model = fit(5, model, loss_func, opt, train_dl, lossBest=None)

epoch:0, train_loss:1.0616543284042286
epoch:1, train_loss:1.0658719638169138
epoch:2, train_loss:0.9775271939693905
epoch:3, train_loss:1.0060471941213138
epoch:4, train_loss:1.0001072941419205
Training completed in 111.2577314376831s


In [56]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            }, 'Models/chk13_softmax')

In [116]:
k=10

In [55]:
# validation MAP
pred_train_gen = PredictGenerator2(Ids_train.Imgs.tolist(),transform_test,TTASize, 4,True)
pred_val_gen = PredictGenerator2(Ids_val.Imgs.tolist(),transform_test,TTASize, 2,True)

feature_train = predict(model,pred_train_gen)
feature_val = predict(model,pred_val_gen)

feature_train = feature_train.reshape(Ids_train.shape[0],TTASize,outDim)
feature_val = feature_val.reshape(Ids_val.shape[0],TTASize,outDim)

predicts = loop_distance(feature_train,feature_val,distanceFun_np,aggFun,returnValue=False,k=5)

MAP(labels,predicts)

0.7370748299319727

In [41]:
temp = torch.ones((batchSize,batchSize),device='cuda:0') * 7
temp[torch.arange(batchSize),torch.arange(batchSize)] = 9

In [42]:
# adv training
feature_train2 = feature_train[Ids_train.length2>1]
predicts2 = loop_distance(feature_train2,feature_train2,distanceFun_np,aggFun,returnValue=False,k=k)
distDict = {i:[j for j in item.tolist() if i!=j] for i,item in enumerate(predicts2)}
batch_sampler = AdvSample(distDict,numBatch,batchSize,maxImg)

gen_train = TripletGenerator(Ids_train2,newWhale_train2,transform)
train_dl= DataLoader(gen_train,num_workers=3,batch_sampler=batch_sampler)

In [43]:
loss_func = loss_func_generator_softmax_Temp(batchSize,distanceFun,temp)

In [44]:
model = fit(5, model, loss_func, opt, train_dl, lossBest=None)

epoch:0, train_loss:0.914326873307671
epoch:1, train_loss:0.9290357761777164
epoch:2, train_loss:0.8302613088830573
epoch:3, train_loss:0.8555441172233696
epoch:4, train_loss:0.833375633210758
Training completed in 111.1541588306427s


In [109]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            }, 'Models/chk13_softmax')